## 2026 EY AI & Data Challenge - Landsat Data Extraction Notebook

This notebook demonstrates Landsat data extraction and the creation of an output file to be used by the benchmark notebook. The baseline data is [Landsat Collection 2 Level 2](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2) data from the MS Planetary Computer catalog.

**Caution**... This notebook requires significant execution time as there are 9,319 data points (unique locations and times) used for data extraction from the Landsat archive. The code takes about 7 hours to run to completion on a typical laptop computer with a typical internet connection. Lower execution times are likely possible with optimization of the data extraction process and the use of cloud computing services.


### Load In Dependencies
The following code installs the required Python libraries (found in the requirements.txt file) in the Snowflake environment to allow successful execution of the remaining notebook code. After running this code for the first time, it is required to “restart” the kernal so the Python libraries are available in the environment. This is done by selecting the “Connected” menu above the notebook (next to “Run all”) and selecting the “restart kernal” link. Subsequent runs of the notebook do not require this “restart” process. 

In [ ]:
!pip install uv
!uv pip install  -r requirements.txt 

In [1]:
import snowflake
from snowflake.snowpark.context import get_active_session
session = get_active_session()

import warnings
warnings.filterwarnings("ignore")

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

from datetime import date
from tqdm import tqdm
import os
import time
import re

tqdm.pandas()  

### Extracting Landsat Data Using API Calls

The API-based method allows us to efficiently access **Landsat** data for specific coordinates and time periods, ensuring scalability and reproducibility of the process.

Through the API, we can query individual bands or compute indices like **NDMI** on the fly. This approach reduces storage requirements and simplifies data preprocessing, making it ideal for large-scale environmental and water quality analysis.

The **compute_Landsat_values** function extracts Landsat surface reflectance values for specific sampling locations using a 100 m focal buffer around each point. For each location:

- A bounding box (bbox) is created around the latitude and longitude coordinates.
- The Microsoft Planetary Computer API is queried for Landsat-8 Level-2 surface reflectance imagery within the date range.
- The nearest low-cloud (<10% cloud cover) scene is selected, and the specified bands (**green**, **nir08**, **swir16**, **swir22**) are loaded.
- Median values of the pixels within the bounding box are computed to reduce the effect of noise or outliers.

**Why the buffer value is 0.00089831**

We want a ~100 m buffer around each point.  
At the equator, 1 degree ≈ 110 km.

Therefore, the degree equivalent of 100 m is:

*buffer_deg ≈ 100 m / 110,000 m per degree ≈ 0.00089831*

This value ensures that the buffer approximately matches the pixel resolution of Landsat imagery, capturing a ~100 m area around each sampling location.


In [2]:
# 原版的code
# # Setup
# tqdm.pandas()

# def compute_Landsat_values(row):
#     lat = row['Latitude']
#     lon = row['Longitude']
#     date = pd.to_datetime(row['Sample Date'], dayfirst=True, errors='coerce')

#     # Buffer size for ~100m 
#     bbox_size = 0.00089831  
#     bbox = [
#         lon - bbox_size / 2,
#         lat - bbox_size / 2,
#         lon + bbox_size / 2,
#         lat + bbox_size / 2
#     ]

#     catalog = pystac_client.Client.open(
#         "https://planetarycomputer.microsoft.com/api/stac/v1",
#         modifier=pc.sign_inplace,
#     )

#     # Wider search range, we'll filter to nearest date later
#     search = catalog.search(
#         collections=["landsat-c2-l2"],
#         bbox=bbox,
#         datetime="2011-01-01/2015-12-31",
#         query={"eo:cloud_cover": {"lt": 10}},
#     )
    
#     items = search.item_collection()

#     if not items:
#         return pd.Series({
#             "nir": np.nan, "green": np.nan, "swir16": np.nan, "swir22": np.nan
#         })

#     try:
#         # Convert sample date to UTC
#         sample_date_utc = date.tz_localize("UTC") if date.tzinfo is None else date.tz_convert("UTC")

#         # Pick the item closest to the sample date
#         items = sorted(
#             items,
#             key=lambda x: abs(pd.to_datetime(x.properties["datetime"]).tz_convert("UTC") - sample_date_utc)
#         )
#         selected_item = pc.sign(items[0])

#         # Load required bands
#         bands_of_interest = ["green", "nir08", "swir16", "swir22"]
#         data = stac_load([selected_item], bands=bands_of_interest, bbox=bbox).isel(time=0)

#         green = data["green"].astype("float")
#         nir = data["nir08"].astype("float")
#         swir16 = data["swir16"].astype("float")
#         swir22 = data["swir22"].astype("float")
        
#         # Compute medians
#         median_green = float(green.median(skipna=True).values)
#         median_nir = float(nir.median(skipna=True).values)
#         median_swir16 = float(swir16.median(skipna=True).values)
#         median_swir22 = float(swir22.median(skipna=True).values)

#         # Replace 0 with NaN
#         median_green = median_green if median_green != 0 else np.nan
#         median_nir = median_nir if median_nir != 0 else np.nan
#         median_swir16 = median_swir16 if median_swir16 != 0 else np.nan
#         median_swir22 = median_swir22 if median_swir22 != 0 else np.nan
        
#         return pd.Series({
#             "nir": median_nir,
#             "green": median_green,
#             "swir16": median_swir16,
#             "swir22": median_swir22,
#         })
    
#     except Exception as e:
#         return pd.Series({
#             "nir": np.nan, "green": np.nan, "swir16": np.nan, "swir22": np.nan
#         })

### Extracting features for the training dataset

In [3]:
Water_Quality_df=pd.read_csv('water_quality_training_dataset.csv')
display(Water_Quality_df.head())

In [4]:
Water_Quality_df.shape

### Note

The Landsat data extraction process for all 9,319 locations typically requires more than 7 hours when executed in a single run. During long executions, you may occasionally encounter API limits, timeout errors, or request failures. To avoid these interruptions, we recommend running the extraction in smaller batches.

In this notebook, we provide a sample code snippet demonstrating how to extract data for the first 200 locations. Participants are encouraged to follow the same batching approach to extract data for all 9,319 locations safely and efficiently.

We have already executed the full extraction for all 9,319 locations and saved the output to **landsat_features_training.csv**, which will be used in the benchmark notebook.  
Similarly, participants can extract Landsat features in batches, combine the batch outputs, and save the final merged dataset as **landsat_features_training.csv** to ensure the benchmark notebook runs smoothly.


### API撈取Landsat的raw檔（官方資料版）

### API撈取Landsat的raw檔（魔改加sample版）

In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace,
)

def compute_Landsat_raw(row):
    cols = ["blue", "green", "red", "nir", "swir16", "swir22", "st_b10"]
    try:
        lat, lon = row['Latitude'], row['Longitude']
        sample_date = pd.to_datetime(row['Sample Date'], dayfirst=True, errors='coerce')
        if pd.isna(sample_date): return pd.Series({k: None for k in cols})
            
        sample_date_utc = sample_date.tz_localize("UTC") if sample_date.tzinfo is None else sample_date.tz_convert("UTC")
        bbox_size = 0.00089831
        bbox = [lon - bbox_size/2, lat - bbox_size/2, lon + bbox_size/2, lat + bbox_size/2]

        search = catalog.search(
            collections=["landsat-c2-l2"],
            bbox=bbox,
            datetime="2011-01-01/2015-12-31",
            query={"eo:cloud_cover": {"lt": 10}},
        )
        items = search.item_collection()
        if not items: return pd.Series({k: None for k in cols})

        selected_item = min(items, key=lambda x: abs(pd.to_datetime(x.properties["datetime"]).tz_convert("UTC") - sample_date_utc))
        target_bands = ["blue", "green", "red", "nir08", "swir16", "swir22", "lwir11"]
        available_bands = [b for b in target_bands if b in selected_item.assets]

        data = stac_load([selected_item], bands=available_bands, bbox=bbox, chunks={}).isel(time=0)
        get_val = lambda b: data[b].values.flatten().tolist() if b in data else None

        return pd.Series({
            "blue": get_val("blue"), "green": get_val("green"), "red": get_val("red"),
            "nir": get_val("nir08"), "swir16": get_val("swir16"), "swir22": get_val("swir22"),
            "st_b10": get_val("lwir11")
        })
    except Exception:
        return pd.Series({k: None for k in cols})

def run_landsat_raw_in_batches(df, out_dir, batch_size=100, start_batch=0, pause_every=5, pause_seconds=2):
    os.makedirs(out_dir, exist_ok=True)
    points = df[['Latitude', 'Longitude', 'Sample Date']].drop_duplicates().reset_index(drop=True)
    n = len(points)
    n_batches = (n + batch_size - 1) // batch_size
    
    print(f"Total points: {n}, Batches: {n_batches}")

    for b in range(start_batch, n_batches):
        out_path = f"{out_dir}/landsat_raw_batch_{b:05d}.parquet"
        s, e = b * batch_size, min((b + 1) * batch_size, n)
        
        print(f"🚀 Processing batch {b}/{n_batches-1} (rows {s}..{e-1})")
        
        batch_points = points.iloc[s:e].reset_index(drop=True)
        batch_raw = batch_points.progress_apply(compute_Landsat_raw, axis=1)
        
        pd.concat([batch_points, batch_raw], axis=1).to_parquet(out_path, index=False)
        
        if pause_every and (b - start_batch + 1) % pause_every == 0:
            time.sleep(pause_seconds)

    print("🎉 Done.")

def merge_landsat_batches(out_dir, merged_path):
    files = sorted([os.path.join(out_dir, f) for f in os.listdir(out_dir) if re.match(r"^landsat_raw_batch_\d{5}\.parquet$", f)])
    df_all = pd.concat([pd.read_parquet(f) for f in files], ignore_index=True)
    df_all.to_parquet(merged_path, index=False)
    print(f"✅ Merged {df_all.shape}")
    return df_all


In [ ]:
run_landsat_raw_in_batches(
    df=Water_Quality_df,
    out_dir="landsat_raw_batches",
    batch_size=100,
    start_batch=0 
    )
    
merge_landsat_batches(
    out_dir="landsat_raw_batches",
    merged_path="landsat_raw_all_v2.parquet"
)

### 儲存landsat的raw檔，轉存github

In [ ]:
landsat_raw_all = merge_landsat_batches(
    out_dir="landsat_raw_batches",
    merged_path="/tmp/landsat_raw_all_v2.parquet"
)

In [ ]:
session.sql("""
    PUT file:///tmp/landsat_raw_all_v2.parquet
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

### 清理掉所有用API撈取landsat的raw檔的暫存

In [ ]:
import os

def clear_landsat_batch_dir(out_dir="landsat_raw_batches"):
    if not os.path.exists(out_dir):
        print(f"📂 Directory does not exist: {out_dir}")
        return

    files = os.listdir(out_dir)

    if not files:
        print(f"📂 Directory already empty: {out_dir}")
        return

    for f in files:
        path = os.path.join(out_dir, f)
        if os.path.isfile(path):
            os.remove(path)

    print(f"🧹 Cleared all files in: {out_dir}")
    
clear_landsat_batch_dir("landsat_raw_batches")

### 用raw檔做各種計算，以median為例

In [ ]:
def clean_pixel_median(val):
    """
    處理單一儲存格的像素資料，計算中位數並排除 0 值。
    """
    try:
        if val is None:
            return np.nan

        # 1. 統一轉換為 numpy array
        if np.isscalar(val):
            if pd.isna(val): return np.nan
            arr = np.array([float(val)])
        elif isinstance(val, str):
            s = val.strip("[]' ").replace(',', ' ')
            if not s: return np.nan
            arr = np.fromstring(s, sep=' ')
        else:
            arr = np.asarray(val, dtype=float)

        # 2. 計算中位數
        if arr.size == 0:
            return np.nan
            
        med = float(np.nanmedian(arr))
        
        # 3. 排除 0 的特殊邏輯
        return med if med != 0 else np.nan
    except Exception:
        return np.nan

def calculate_spectral_indices(df, prefix="med_", eps=1e-10):
    """
    根據中位數波段計算遙測指數。
    """
    # 為了簡化程式碼，先建立捷徑變數
    b = df[f'{prefix}blue']
    g = df[f'{prefix}green']
    r = df[f'{prefix}red']
    n = df[f'{prefix}nir']
    s16 = df[f'{prefix}swir16']
    
    # 計算各項指數
    df[f'{prefix}NDMI']  = (n - s16) / (n + s16 + eps)
    df[f'{prefix}MNDWI'] = (g - s16) / (g + s16 + eps)
    df[f'{prefix}NDVI']  = (n - r)   / (n + r + eps)
    df[f'{prefix}NDTI']  = (r - g)   / (r + g + eps)
    df[f'{prefix}EVI']   = 2.5 * (n - r) / (n + 6*r - 7.5*b + 1 + eps)
    
    return df

def process_landsat_pipeline(raw_df, band_cols=None):
    """
    主處理流程：清洗波段 -> 計算中位數 -> 計算指數。
    """
    if band_cols is None:
        band_cols = ['blue', 'green', 'red', 'nir', 'swir16', 'swir22']
        
    print(f"--- 1. 提取原始欄位 ---")
    features_df = raw_df[['Latitude', 'Longitude', 'Sample Date']].copy()
    
    print(f"--- 2. 計算各波段中位數 ({len(band_cols)} bands) ---")
    for col in band_cols:
        if col in raw_df.columns:
            features_df[f"med_{col}"] = raw_df[col].apply(clean_pixel_median)
            print(f"   Done: med_{col}")
            
    print(f"--- 3. 計算光譜指數 ---")
    features_df = calculate_spectral_indices(features_df)
    
    print("--- 處理完成 ---")
    return features_df

In [ ]:
# ==========================================
# 實際執行 (Main Execution)
# ==========================================

# 1. 載入資料
path = "landsat_raw.parquet"
landsat_raw = pd.read_parquet(path)


# 2. 執行 Pipeline
landsat_train_features = process_landsat_pipeline(landsat_raw)

# 3. 儲存結果
output_path = "/tmp/landsat_features_training_jin.csv"
landsat_train_features.to_csv(output_path, index=False)
print(f"結果已儲存至: {output_path}")

# 7) 上傳至 Snowflake Stage
session.sql(f"""
    PUT file://{output_path}
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved and uploaded! Refresh the browser to see the files in the sidebar.")

# 預覽結果
landsat_train_features.head()


In [ ]:
##原本官方的code
# # Extract band values from Landsat for training dataset
# train_features_path = "landsat_features_training.csv"

# print("🚀 Running Landsat feature extraction for training data...")
# landsat_train_features = Water_Quality_df.progress_apply(compute_Landsat_values, axis=1)
# landsat_train_features.to_csv(train_features_path, index=False)

**NDMI and MNDWI Indices**

In this notebook, we compute two commonly used water-related indices from the extracted Landsat bands:

- **NDMI (Normalized Difference Moisture Index):**  
  Measures vegetation water content and surface moisture.  
  Computed as *(NIR - SWIR16) / (NIR + SWIR16)*.

- **MNDWI (Modified Normalized Difference Water Index):**  
  Highlights open water features by enhancing water reflectance and suppressing built-up areas.  
  Computed as *(Green - SWIR16) / (Green + SWIR16)*.

An **epsilon value** (*eps = 1e-10*) is added to the denominators to avoid division by zero.  
These indices are widely used in hydrological and water quality analyses for detecting water presence and vegetation moisture levels.


In [8]:
landsat_train_features['Latitude'] = Water_Quality_df['Latitude']
landsat_train_features['Longitude'] = Water_Quality_df['Longitude']
landsat_train_features['Sample Date'] = Water_Quality_df['Sample Date']
landsat_train_features = landsat_train_features[['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']]

In [ ]:
# Preview File
landsat_train_features.head()

**Note:** If you're using your own workspace, remember to replace "EY-AI-and-Data-Challenge" with your workspace name in the file path.

### Extracting features for the validation dataset

In [11]:
Validation_df=pd.read_csv('submission_template.csv')
display(Validation_df.head())

In [12]:
Validation_df.shape
Validation_df_20 = Validation_df.iloc[:20].copy()


In [ ]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=pc.sign_inplace,
)

def compute_Landsat_raw(row):
    cols = ["blue", "green", "red", "nir", "swir16", "swir22", "st_b10"]
    try:
        lat, lon = row['Latitude'], row['Longitude']
        sample_date = pd.to_datetime(row['Sample Date'], dayfirst=True, errors='coerce')
        if pd.isna(sample_date): return pd.Series({k: None for k in cols})
            
        sample_date_utc = sample_date.tz_localize("UTC") if sample_date.tzinfo is None else sample_date.tz_convert("UTC")
        bbox_size = 0.00089831
        bbox = [lon - bbox_size/2, lat - bbox_size/2, lon + bbox_size/2, lat + bbox_size/2]

        search = catalog.search(
            collections=["landsat-c2-l2"],
            bbox=bbox,
            datetime="2011-01-01/2015-12-31",
            query={"eo:cloud_cover": {"lt": 10}},
        )
        items = search.item_collection()
        if not items: return pd.Series({k: None for k in cols})

        selected_item = min(items, key=lambda x: abs(pd.to_datetime(x.properties["datetime"]).tz_convert("UTC") - sample_date_utc))
        target_bands = ["blue", "green", "red", "nir08", "swir16", "swir22", "lwir11"]
        available_bands = [b for b in target_bands if b in selected_item.assets]

        data = stac_load([selected_item], bands=available_bands, bbox=bbox, chunks={}).isel(time=0)
        get_val = lambda b: data[b].values.flatten().tolist() if b in data else None

        return pd.Series({
            "blue": get_val("blue"), "green": get_val("green"), "red": get_val("red"),
            "nir": get_val("nir08"), "swir16": get_val("swir16"), "swir22": get_val("swir22"),
            "st_b10": get_val("lwir11")
        })
    except Exception:
        return pd.Series({k: None for k in cols})

def run_landsat_raw_in_batches(df, out_dir, batch_size=100, start_batch=0, pause_every=5, pause_seconds=2):
    os.makedirs(out_dir, exist_ok=True)
    points = df[['Latitude', 'Longitude', 'Sample Date']].drop_duplicates().reset_index(drop=True)
    n = len(points)
    n_batches = (n + batch_size - 1) // batch_size
    
    print(f"Total points: {n}, Batches: {n_batches}")

    for b in range(start_batch, n_batches):
        out_path = f"{out_dir}/landsat_raw_batch_{b:05d}.parquet"
        s, e = b * batch_size, min((b + 1) * batch_size, n)
        
        print(f"🚀 Processing batch {b}/{n_batches-1} (rows {s}..{e-1})")
        
        batch_points = points.iloc[s:e].reset_index(drop=True)
        batch_raw = batch_points.progress_apply(compute_Landsat_raw, axis=1)
        
        pd.concat([batch_points, batch_raw], axis=1).to_parquet(out_path, index=False)
        
        if pause_every and (b - start_batch + 1) % pause_every == 0:
            time.sleep(pause_seconds)

    print("🎉 Done.")

def merge_landsat_batches(out_dir, merged_path):
    files = sorted([os.path.join(out_dir, f) for f in os.listdir(out_dir) if re.match(r"^landsat_raw_batch_\d{5}\.parquet$", f)])
    df_all = pd.concat([pd.read_parquet(f) for f in files], ignore_index=True)
    df_all.to_parquet(merged_path, index=False)
    print(f"✅ Merged {df_all.shape}")
    return df_all

# -----------------------------
# 執行
# -----------------------------
run_landsat_raw_in_batches(
        df=Validation_df,
        out_dir="landsat_validation_raw_batches",
        batch_size=100,
        start_batch=0 
    )
    
merge_landsat_batches(
        out_dir="landsat_validation_raw_batches",
        merged_path="landsat_validation_raw_all_v2.parquet"
    )

In [ ]:
merge_landsat_batches(
        out_dir="landsat_validation_raw_batches",
        merged_path="/tmp/landsat_validation_raw_all_v2.parquet"
    )
session.sql("""
    PUT file:///tmp/landsat_validation_raw_all_v2.parquet
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

In [ ]:
# 1. 載入資料
landsat_validation_raw = pd.read_parquet("landsat_validation_raw_all.parquet")
landsat_validation_raw 

In [ ]:
# 1. 載入資料
landsat_validation_raw = pd.read_parquet("landsat_validation_raw_all.parquet")

# 2. 執行 Pipeline
landsat_validation_features = process_landsat_pipeline(landsat_validation_raw)

# 3. 儲存結果
output_path = "/tmp/landsat_features_validation_jin.csv"
landsat_validation_features.to_csv(output_path, index=False)
print(f"結果已儲存至: {output_path}")

# 7) 上傳至 Snowflake Stage
session.sql(f"""
    PUT file://{output_path}
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved and uploaded! Refresh the browser to see the files in the sidebar.")

# 預覽結果
landsat_validation_features.head()


In [ ]:
# 5) PUT 到 Snowflake workspace
session.sql(f"""
    PUT file:///tmp/landsat_validation_raw_all_v2.parquet
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

In [ ]:
path = "/tmp/landsat_validation_raw_all.parquet"
landsat_val_raw = pd.read_parquet(path)

def compute_median_from_pixels(arr):
    # median(skipna=True) + median==0 -> NaN
    if arr is None:
        return np.nan
    a = np.array(arr, dtype="float")
    a = a[~np.isnan(a)]
    if len(a) == 0:
        return np.nan
    median_val = float(np.median(a))
    return median_val if median_val != 0 else np.nan

# 0) 先看一下有哪些欄位（debug 用）
print("Columns:", list(landsat_val_raw.columns))

# 1) 建立輸出表：先放 key 欄位
landsat_val_features = landsat_val_raw[['Latitude', 'Longitude', 'Sample Date']].copy()

# 2) 自動對應 band 欄位名稱（避免你檔案裡不是 'nir' 而是 'nir08' 之類）
col_map_candidates = {
    "nir":    ["nir", "nir08", "B5", "SR_B5"],
    "green":  ["green", "B3", "SR_B3"],
    "swir16": ["swir16", "swir1", "B6", "SR_B6"],
    "swir22": ["swir22", "swir2", "B7", "SR_B7"],
}

resolved = {}
for out_name, candidates in col_map_candidates.items():
    found = next((c for c in candidates if c in landsat_val_raw.columns), None)
    resolved[out_name] = found

missing = [k for k, v in resolved.items() if v is None]
if missing:
    raise KeyError(
        f"These expected bands are missing in parquet: {missing}. "
        f"Available columns: {list(landsat_val_raw.columns)}"
    )

print("Resolved band columns:", resolved)

# 3) median 計算：從 landsat_val_raw 的原始像素陣列欄位算出來，寫入 features
for out_name, raw_col in resolved.items():
    landsat_val_features[out_name] = landsat_val_raw[raw_col].apply(compute_median_from_pixels)


In [14]:
# Create indices: NDMI and MNDWI
eps = 1e-10
landsat_val_features['NDMI'] = (landsat_val_features['nir'] - landsat_val_features['swir16']) / (landsat_val_features['nir'] + landsat_val_features['swir16'])
landsat_val_features['MNDWI'] = (landsat_val_features['green'] - landsat_val_features['swir16']) / (landsat_val_features['green'] + landsat_val_features['swir16'] + eps)

In [15]:
landsat_val_features['Latitude'] = Validation_df['Latitude']
landsat_val_features['Longitude'] = Validation_df['Longitude']
landsat_val_features['Sample Date'] = Validation_df['Sample Date']
landsat_val_features = landsat_val_features[['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']]

In [ ]:
# Preview File
landsat_val_features.head()
landsat_val_features.to_csv("/tmp/landsat_features_validation_jin.csv",index = False)

In [ ]:
session.sql("""
    PUT file:///tmp/landsat_features_validation_jin.csv
    'snow://workspace/USER$.PUBLIC."EY-AI-and-Data-Challenge"/versions/live/'
    AUTO_COMPRESS=FALSE
    OVERWRITE=TRUE
""").collect()

print("File saved! Refresh the browser to see the files in the sidebar")

**Note:** If you're using your own workspace, remember to replace "EY-AI-and-Data-Challenge" with your workspace name in the file path.